In [3]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('hospitalizacion_modelado.csv')
df.head()

# voy a observar que sucede si no tomo dias complicacion (la definicion es algo ambigua y no se entiende de donde sale)
# ademas de que al parecer overfitea el modelo


,muestrasTomadas,diabetes,hospitalizacionUltimoMes,antibioticoProfilaxis_OROQUINOLONAS,itu,tipoCultivo_HEMOCULTIVO,tipoCultivo_HEMOCULTIVO Y UROCULTIVO,tipoCultivo_UROCULTIVO,agenteAislado_E.COLI,agenteAislado_PSEUDOMONAS AERUGINOSA,"patronResistencia_ AMPI R, CIPRO R, GENTA R, SULFA M R",patronResistencia_MULTI SENSIBLE,"patronResistencia_RESISTENTE A AMPI, CIPRO Y GENTA","patronResistencia_RESISTENTE A AMPI, SULFA, CEFADROXILO, CEFUROXIMO, CIPRO Y CEFEPIME, CEFOTAXIMA",hospitalizacion,psa,antibioticoProfilaxis_OTROS,tipoCultivo_NO,agenteAislado_NO,patronResistencia_NO
0,12.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.386294,0,1,1,1
1,12.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2.041220,0,1,1,1
2,24.0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,3.367296,0,0,0,0
3,12.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.945910,0,1,1,1
4,24.0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,3.367296,0,0,0,1


In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# voy a escalar estas variables para evitar sesgos en el modelo
df['muestrasTomadas'] = scaler.fit_transform(np.reshape(df['muestrasTomadas'],(-1,1))).ravel()
df['psa'] = scaler.fit_transform(np.reshape(df['psa'],(-1,1))).ravel()


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523 entries, 0 to 522
Data columns (total 20 columns):
 #   Column                                                                                             Non-Null Count  Dtype  
---  ------                                                                                             --------------  -----  
 0   muestrasTomadas                                                                                    523 non-null    float64
 1   diabetes                                                                                           523 non-null    int64  
 2   hospitalizacionUltimoMes                                                                           523 non-null    int64  
 3   antibioticoProfilaxis_OROQUINOLONAS                                                                523 non-null    int64  
 4   itu                                                                                                523 non-null    int64  

In [7]:
x=[]
for i in df:
    if (i!='hospitalizacion'):
        x.append(np.array(df[i]).reshape(-1,1))

X=np.hstack(x)
y=np.array(df.hospitalizacion).reshape(-1,1)

In [8]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(sampling_strategy='not majority',random_state=42)

X , y = ros.fit_resample(X,y)

# verificar el oversampling
print('value_counts y:',np.unique(y,return_counts=True))
print('len X:',len(X))

value_counts y: (array([0, 1]), array([500, 500]))
len X: 1000


In [9]:
'''from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=15,n_iter=2,random_state=42)
X=svd.fit_transform(X)'''

'from sklearn.decomposition import TruncatedSVD\nsvd=TruncatedSVD(n_components=15,n_iter=2,random_state=42)\nX=svd.fit_transform(X)'

In [10]:
from sklearn.model_selection import train_test_split

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
from sklearn.linear_model import Perceptron


In [12]:
# para elegir un buen alpha voy a probar los parametros con GridSearchCV

from sklearn.model_selection import GridSearchCV

neurona = Perceptron()

neurona_params = {'alpha':[0.1,0.001,0.0001,0.00001,0.000001],
                  'penalty':['l2','l1','elasticnet']}

modelo_selector_neurona = GridSearchCV(neurona,param_grid=neurona_params,cv=5)

modelo_selector_neurona.fit(X,y)


GridSearchCV(cv=5, estimator=Perceptron(),
             param_grid={'alpha': [0.1, 0.001, 0.0001, 1e-05, 1e-06],
                         'penalty': ['l2', 'l1', 'elasticnet']})

In [13]:
modelo_selector_neurona.best_params_

{'alpha': 0.001, 'penalty': 'elasticnet'}

In [14]:
y_pred = modelo_selector_neurona.predict(X)

In [15]:
from sklearn.metrics import classification_report
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.39      0.56       500
           1       0.62      1.00      0.77       500

    accuracy                           0.69      1000
   macro avg       0.81      0.69      0.66      1000
weighted avg       0.81      0.69      0.66      1000



In [16]:
# el modelo está un poco debil, por lo que voy a usar una red neuronal
from sklearn.neural_network import MLPClassifier

total_capa_neuronas = (200,200,200) # tamaño de la red neuronal

red_neuronal = MLPClassifier(hidden_layer_sizes=total_capa_neuronas,activation='logistic',alpha=0.001,solver='adam',max_iter=500)

red_neuronal.fit(X,y)

MLPClassifier(activation='logistic', alpha=0.001,
              hidden_layer_sizes=(200, 200, 200), max_iter=500)

In [17]:
red_neuronal.predict([X[3]])

array([0])

In [18]:
y[3]

1

In [19]:
y_pred=red_neuronal.predict(X)

In [20]:
# podemos decir que el modelo ha mejorado considerablemente
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76       500
           1       0.76      0.76      0.76       500

    accuracy                           0.76      1000
   macro avg       0.76      0.76      0.76      1000
weighted avg       0.76      0.76      0.76      1000



In [21]:
# voy a probar hacerlo en un GridSearchCV

red_prueba = MLPClassifier()

red_params = {'alpha':[0.1,0.01,0.001,0.0001,0.00001,0.000001],
              'hidden_layer_sizes':[(100,100,100)],
              'max_iter':[400]
              }

red_modelo_prueba = GridSearchCV(red_prueba,param_grid=red_params,cv=5)

red_modelo_prueba.fit(X,y)

GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'alpha': [0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06],
                         'hidden_layer_sizes': [(100, 100, 100)],
                         'max_iter': [400]})

In [22]:
red_modelo_prueba.best_params_

{'alpha': 1e-05, 'hidden_layer_sizes': (100, 100, 100), 'max_iter': 400}

In [23]:
y_pred = red_modelo_prueba.predict(X)

In [24]:
# se puede observar que incluso con unas pocas iteraciones el modelo mejoro muy considerablemente
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.84      0.89       500
           1       0.85      0.95      0.90       500

    accuracy                           0.89      1000
   macro avg       0.90      0.89      0.89      1000
weighted avg       0.90      0.89      0.89      1000



In [25]:
red_modelo_prueba.predict([X[1]])

array([1])

In [26]:
y[1]

1